# Mengimport Library yang Dibutuhkan

In [21]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as stopwords_scratch
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji

# Membuat dataframe stopwords.

In [23]:
# Panggil stopword ID
list_stopwords = stopwords_scratch.words('indonesian')
# Panggil stopword EN
list_stopwords_en = stopwords_scratch.words('english')
# Gabungkan ID & EN
list_stopwords.extend(list_stopwords_en)
# Tambah daftar stopword jika perlu
list_stopwords.extend(['ya', 'yg', 'ga', 'aja', 'yuk', 'dah', 'ngga', 'engga', 'ygy', 'gak', 'nya', 'baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'])

# Buat DataFrame dari list stopwords
stopwords_df = pd.DataFrame(list_stopwords, columns=['stopword'])

# Simpan DataFrame sebagai CSV
stopwords_df.to_csv('./data/stopwords.csv', index=False, header=False)

# Mengimport dataframe yang dibutuhkan.

In [24]:
comment_df = pd.read_csv("./data/dataset_comment_yt.csv")
stopword_df = pd.read_csv("./data/stopwords.csv", header=None)

# Menginisialiasi Stemmer untuk bahasa Indonesia (mengubah kata ke bentuk dasar).

In [7]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membuat method pre processing text untuk membersihkan dataset dari emoji, karakter khusus, membuat jadi lower case.

In [13]:
# Fungsi pre-processing
def preprocess_text(text):
    # Hapus emoji
    text = emoji.replace_emoji(text, replace="")
    
    # Hapus nama orang (pola kapitalisasi)
    text = re.sub(r'\b[A-Z][a-z]*\b', '', text)
    
    # Hapus karakter khusus, angka, dan URL
    text = re.sub(r'http\S+|www\S+|https\S+|[^a-zA-Z\s]', '', text)
    
    # Konversi ke lowercase
    text = text.lower()
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopword_df]
    
    # Remove very short words (kurang dari 2 karakter)
    tokens = [word for word in tokens if len(word) > 2]
    
    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Gabungkan kembali token menjadi string
    return ' '.join(tokens)


# Menerapkan method preprocess_text ke dalam df pada kolom 'comment' yang disimpan menjadi kolom 'cleaned_comment'

In [17]:
comment_df['comment'] = comment_df['comment'].astype(str)
comment_df['cleaned_comment'] = comment_df['comment'].apply(preprocess_text)

comment_df = comment_df[comment_df['cleaned_comment'].str.strip() != '']

# Tampilkan hasil pre-processing pada beberapa baris pertama
print(comment_df[['comment', 'cleaned_comment']].head())

                                              comment  \
4                             Gibraaaann MANTAPKU🔥🔥🔥🔥   
5   Pengen balik ke sini.  Lihat orang2 yg dulu ng...   
8             Woi Tapera Tapera ok gas ok gas pretttt   
9                               AkademisiRocky Gerung   
10  uustadzRockyvGerungprofDrAhliFilsapatnterkemuk...   

                                      cleaned_comment  
4                                              mantap  
5   balik ke sini orang yg dulu ngalem bocil msh t...  
8                               ok gas ok gas pretttt  
9                                      akademisirocky  
10  uustadzrockyvgerungprofdrahlifilsapatnterkemukadi  


# Menghapus data duplikat yang ada di kolom 'cleaned_comment'

In [18]:
print("Jumlah data duplikat:", comment_df['cleaned_comment'].duplicated().sum())

comment_df = comment_df.drop_duplicates(subset='cleaned_comment')

Jumlah data duplikat: 1719


# Menyimpan data yang telah dibersihkan ke dalam bentuk file CSV.

In [20]:
comment_df.to_csv('./data/cleaned_comment_yt.csv', index=False)